In [ ]:
import pandas as pd
import yaml
from pathlib import Path

config_path = Path("../config.yaml")

with open(config_path, "r") as f:
    config = yaml.safe_load(f)

csv_relative_path = config["data"]["raw_data_csv"]["raw_data"]

csv_path = config_path.parent / csv_relative_path

df = pd.read_csv(csv_path)
df

In [ ]:
#https://www.kaggle.com/datasets/nehalbirla/vehicle-dataset-from-cardekho?select=car+details+v4.csv

In [ ]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(drop='first', sparse_output=False)
encoded = encoder.fit_transform(df[['fuel', 'seller_type', 'transmission']])

encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(['fuel', 'seller_type', 'transmission']))
df = pd.concat([df.drop(['fuel', 'seller_type', 'transmission'], axis=1), encoded_df], axis=1)

In [ ]:
df['brand'] = df['name'].str.split().str[0]
df.drop('name', axis=1, inplace=True)


encoder = OneHotEncoder(drop='first', sparse_output=False)
encoded = encoder.fit_transform(df[['brand']])

#Create new DataFrame with encoded brand columns
encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(['brand']), index=df.index)

#Concatenate encoded columns back to original df
df = pd.concat([df.drop('brand', axis=1), encoded_df], axis=1)


In [ ]:
df['car_age'] = 2025 - df['year']
df.drop('year', axis=1, inplace=True)

In [ ]:
df['owner'].unique()

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

encoder = OrdinalEncoder(categories=[['First Owner', 'Second Owner', 'Third Owner', 'Fourth & Above Owner', 'Test Drive Car']])
df[['owner']] = encoder.fit_transform(df[['owner']])

In [ ]:
df['mileage'].unique()

In [ ]:
#df['mileage'] = df['mileage'].str.replace(' kmpl', '', regex=False).astype(float)
df['mileage'] = df['mileage'].str.extract(r'([\d.]+)').astype(float)

In [ ]:
df['engine'] = df['engine'].str.replace(' CC', '', regex=False).astype(float)

In [ ]:
df['max_power'].unique()

In [ ]:
#df['max_power'] = df['max_power'].str.replace(' bhp', '', regex=False).astype(float)
df['max_power'] = pd.to_numeric(df['max_power'].str.replace(' bhp', '', regex=False), errors='coerce')

# Fill NaN with the mean value
df['max_power'] = df['max_power'].fillna(df['max_power'].mean())

In [ ]:
import re

# Extract numeric torque value
df['torque'] = df['torque'].str.extract(r'([\d.]+)').astype(float)

In [ ]:
df.dtypes

In [ ]:
#clean_relative_path = config["data"]["clean_data_csv"]["clean_data"]
#clean_csv_path = config_path.parent / clean_relative_path

# Ensure the directory exists
#clean_csv_path.parent.mkdir(parents=True, exist_ok=True)

# Save the cleaned DataFrame
#df.to_csv(clean_csv_path, index=False)

In [ ]:
df.isnull().sum()

In [ ]:
df = df.dropna(axis=0, how='any')

In [ ]:
df.isnull().sum()

In [ ]:
clean_relative_path = config["data"]["clean_data_csv"]["clean_data"]
clean_csv_path = config_path.parent / clean_relative_path

# Ensure the directory exists
clean_csv_path.parent.mkdir(parents=True, exist_ok=True)

# Save the cleaned DataFrame
df.to_csv(clean_csv_path, index=False)